In [ ]:
import pandas as pd
import numpy as np
import math

# Compute returns

def rates(time_series):
    deltas = np.diff(time_series)
    rates = deltas/time_series[:-1]
    return rates

def mean_rate(times_series):
    return np.mean(rates(times_series))


def portfolio_return(weights, ts1, ts2):
    means = [mean_rate(ts1), mean_rate(ts2)]
    return np.dot(weights, means)

####

def mean_profile(time_series):
    tsl = list(time_series)
    profile = [tsl[0]]
    rate = mean_rate(time_series)
    for i in range(1,len(time_series)):
        last = profile[-1]
        next = (1 + rate)*last 
        profile.append(next)
    return np.array(profile)

# Normalization (recenter)

def normalize(time_series):
    return time_series - np.mean(time_series)

def risk(time_series):
    mean_ = np.mean(time_series)
    var_  = np.var(time_series)
    risk_ = math.sqrt(var_)/mean_
    return risk_


# Off-diagonal term in covariance matrix

def cov2(ts1, ts2):
    nts1 = normalize(ts1)
    nts2 = normalize(ts2)
    n = len(nts1)
    cov_ = np.sum(nts1*nts2)/n
    return cov_

# Covariance matrix

def covariance(ts1, ts2):
    s11 = np.var(ts1)
    s22 = np.var(ts2)
    s12 = cov2(ts1,ts2)
    return np.array([[s11, s12], [s12, s22]])

def rho(ts1, ts2):
    s11 = np.var(ts1)
    s22 = np.var(ts2)
    s12 = cov2(ts1,ts2)
    return s12/math.sqrt(s11*s22)

# Portfolio variance, risk and relative risk

# portfolio variance is w'Cw, where w is
# viewed as a column vector, w' is its transpose,
# and C is the covariance matrix

# Units: dollars squared
def portfolio_variance(weights, ts1, ts2):
    C = covariance(ts1, ts2)
    return np.dot(weights, np.dot(C, weights))

# Take the square root. 
# Now the units are dollars!
def portfolio_risk(weights, ts1, ts2):
    c = portfolio_variance(weights, ts1, ts2)
    r = math.sqrt(c)
    return round(r,2)

# Units = dollars
def portfolio_mean(weights, ts1, ts2):
    means = [np.mean(ts1), np.mean(ts2)]
    return np.dot(weights, means)

# The relative risk is dimensionless.
# Easier to intepret, IMHO
def portfolio_rel_risk(weights, ts1, ts2):
    pfv = portfolio_variance(weights, ts1, ts2)
    rr = math.sqrt(pfv)/portfolio_mean(weights, ts1, ts2)
    return round(rr,4)

def portfolio_risk_return(weights, ts1, ts2):
    rsk = portfolio_rel_risk(weights, ts1, ts2)
    ret = portfolio_return(weights, ts1, ts2)
    return np.array([rsk, ret])

def pf_rr(w, ts1, ts2):
    return portfolio_risk_return(np.array([w, 1-w]), ts1, ts2)

def print_rr(w, ts1, ts2):
    print(w, pf_rr(w, ts1, ts2))
    
    
w1 = np.array([1.0, 0.0])
w2 = np.array([0.5, 0.5])
w3 = np.array([0.0, 1.0])

In [ ]:
# Import S&P data

dfSP = pd.read_csv('data/^GSPC.csv')
dfSP[0:4]

In [ ]:
# Get the closing prices for each YEAR as a numpy array
sp = dfSP['Close'][1::12].values
sp

In [ ]:
# Import Walmart data
dfWMT = pd.read_csv('data/WMT.csv')
# Get the closing values for each YEAR as an array
wmt = dfWMT['Close'][1::12].values
wmt

In [ ]:
for i in range(0, 11):
    print_rr(i/10, wmt, sp)

In [ ]:
rho(sp, wmt)